# Tổng hợp code từ sources:
1. https://www.kaggle.com/dschettler8845/visual-in-depth-eda-vinbigdata-competition-data

In [6]:
import os
from tqdm import tqdm
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

### Setup global variables

In [3]:
PATH_TRAIN_DATA = 'E:/Kaggle/VinBigData/train/'
PATH_TEST_DATA = 'E:/Kaggle/VinBigData/test/'


## Tổng quan

Dữ liệu được lưu dưới định dạng DICOM

Tập train với 15,000 ảnh

Tập public test với 3,000 ảnh

Ảnh được gắn với 14 nhãn:

0 - Aortic enlargement

1 - Atelectasis

2 - Calcification

3 - Cardiomegaly

4 - Consolidation

5 - ILD

6 - Infiltration

7 - Lung Opacity

8 - Nodule/Mass

9 - Other lesion

10 - Pleural effusion

11 - Pleural thickening

12 - Pneumothorax

13 - Pulmonary fibrosis

14 - "No finding" observation was intended to capture the absence of all findings above

Một lưu ý theo ....

### Dữ liệu mẫu

Mô tả dữ liệu:

train.csv: metadata của tập train, với mỗi hàng là một mẫu ảnh, bao gồm tên nhãn và bounding box (một ảnh có thể có nhiều hàng)

![figures/overview_data](figures/overview_data.PNG)

sample_submission.csv: sample submission file in the correct format
